In [2]:
import pandas as pd
import numpy as np

In [3]:
trans_name="TRANSLATION"
dft=pd.DataFrame()

def create_df():
    test = {'cust_name': ['Boris Johnson','Tony Blair','Matt Hancock','Andy Gove', 'Boris Johnson','Tony Blair'],
        'cust_id': [100,101,102,103,100,103]
        }
    return pd.DataFrame(test, columns = ['cust_name', 'cust_id'])
df=create_df()

In [4]:
def load_translation_table(dft, pickle_name):
    dft= pd.read_pickle(pickle_name)
    return True
     
def save_translation_table(dft, pickle_name):
    dft.to_pickle(pickle_name)
    return True


These functions below are meant to manage the available population of SYNs
If you only need to anonymise one file, a straight process to identify unique keys and associate to a series is fine
But if you are going to apply to multiple separate tables, you need to manage the extra ids that may appear down the line

In [5]:
def create_translation_seq (start,end):
    dft=pd.DataFrame(columns=["syn","live"])
    if start>0:
        l = list(range(start,end+1))
        dfrecs = pd.DataFrame(l, columns=["syn"])
        dfrecs["live"]= np.nan
        dft=dft.append(dfrecs)
    return dft
def expand_available_syns(dft, start, end):
    list_existing= dft.syn.to_list()
    list_new=range(start,end+1)
    list_to_add=list(set(list_new) - set(list_existing))    
    df_extra = pd.DataFrame({"syn":list_to_add,
                    "live":[np.nan for i in list_to_add]})
    print ("Added ",len(df_extra)," elements to the translation table")
    return dft.append(df_extra)



These functions below do the trick of finding a suitable value and replacing it by manually iterating and finding a slot and replacing that in the population and taking into account previuously assigned keuys.
It is a different approach than using dataframe queries and may come handy for scenarios of multiple files, or having to re-anonymise things so that the previous assignments are not changed (e.g. receving a new version of the dataset)

In [6]:
def anonymise_value(live_value,live_column_name,syn_column_name, df, dft ):
    try:
        i=next(iter(dft[dft.live==live_value].index))
    except:
        try: 
            i= dft[pd.isnull(dft['live'])].index.values.astype(int)[0]
        except:   
            print("Ran out of syn ids for ", live_value)
            return False
    dft.at[i,'live']= live_value
    syn_value=dft.iloc[i]["syn"]
    df.loc[df[live_column_name]== live_value, syn_column_name] = syn_value

def anonymise_mass_1 (df,column_name,syn_name,dft):
    live_list = df[column_name].drop_duplicates().to_list()
    df_anon=df.copy()
    for i in live_list:
        anonymise_value(i,column_name,syn_name,df_anon,dft)      
    return df_anon.drop(column_name, axis="columns")

In [7]:
dft= create_translation_seq(1,10)
dft=expand_available_syns(dft,1,101)
df_anon= anonymise_mass_1(df,"cust_id","syn",dft)
df_anon

Added  91  elements to the translation table


,cust_name,syn
0,Boris Johnson,1.0
1,Tony Blair,2.0
2,Matt Hancock,3.0
3,Andy Gove,4.0
4,Boris Johnson,1.0
5,Tony Blair,4.0


In [8]:
def de_anonymise(df,column_name,dft,syn_name):
    return df.merge(dft, left_on = column_name, right_on = syn_name, how='left' ).drop("syn",axis="columns")

de_anonymise(df_anon,"syn",dft,"syn")

,cust_name,live
0,Boris Johnson,100
1,Tony Blair,101
2,Matt Hancock,102
3,Andy Gove,103
4,Boris Johnson,100
5,Tony Blair,103


In [9]:
dft

,syn,live
0,1,100
1,2,101
2,3,102
3,4,103
4,5,NaN
...,...,...
86,97,NaN
87,98,NaN
88,99,NaN
89,100,NaN


In [22]:
# this is not working at all
def anonymise_mass_2(df,column_name,dft,syn_name):
    sdf=set(df[column_name])
    sdft=set(dft["live"])
    ldfnew=list(sdf-sdft)
    l= dft['syn'].to_list()
    if l:
        max_syn=max(l)
        print(max_syn)
        lnew_syn=range(max_syn+1,max_syn+1+len(ldfnew))
        dftnew = pd.DataFrame({"live":ldfnew, "syn":lnew_syn})
        dft=dft.append(dftnew)  
    print(sdf)
    print(sdft)
    print(ldfnew)
    print(list(lnew_syn))
    df_anon=df.merge(dft, left_on = column_name, right_on = syn_name, how='left').drop(column_name,axis="columns") 
    return df_anon

df=create_df()   
dft=create_translation_seq(0,2)
anonymise_mass_2(df,"cust_id",dft,"syn")

{100, 101, 102, 103}
set()
[100, 101, 102, 103]


UnboundLocalError: local variable 'lnew_syn' referenced before assignment

In [10]:
df

,cust_name,cust_id
0,Boris Johnson,100
1,Tony Blair,101
2,Matt Hancock,102
3,Andy Gove,103
4,Boris Johnson,100
5,Tony Blair,103


3